# Statistical Approaches for Territory Alignment
###### Created by : Astajyoti Behera

## Hungarian Algorithm

This implementation takes as input a cost matrix of shape (n, m), where n is the number of rows (i.e., territories) and m is the number of columns (i.e., sales representatives). The output is a list of tuples representing the optimal assignment, where each tuple contains the row and column indices of the assigned cell.

In [ ]:
import numpy as np

def hungarian_algorithm(cost_matrix):
    """
    Solves the linear assignment problem using the Hungarian algorithm.

    Parameters:
    -----------
    cost_matrix : numpy array of shape (n, m)
        The cost matrix for the assignment problem.

    Returns:
    --------
    list of tuples
        A list of (row, col) index pairs representing the optimal assignment.
    """

    # Step 1: Subtract the minimum value of each row from all elements in that row.
    row_mins = np.min(cost_matrix, axis=1)
    cost_matrix -= row_mins.reshape(-1, 1)

    # Step 2: Subtract the minimum value of each column from all elements in that column.
    col_mins = np.min(cost_matrix, axis=0)
    cost_matrix -= col_mins

    # Step 3: Create a mask of 0's and 1's to mark the assigned cells.
    n, m = cost_matrix.shape
    mask = np.zeros((n, m), dtype=bool)

    # Step 4: Iterate until all cells are marked.
    while True:
        # Find the row and column indices of the unmarked cell with the lowest cost.
        unmarked_rows, unmarked_cols = np.nonzero(~mask)
        if not unmarked_rows.size:
            break
        row, col = unmarked_rows[0], unmarked_cols[0]
        for i, j in zip(unmarked_rows, unmarked_cols):
            if cost_matrix[i, j] < cost_matrix[row, col]:
                row, col = i, j

        # Mark the cell with the lowest cost.
        mask[row, :] = True
        mask[:, col] = True

        # Unmark any other cells in the same row or column.
        mask[row, col] = False
        mask[row, np.nonzero(mask[row, :])[0]] = False
        mask[np.nonzero(mask[:, col])[0], col] = False

    # Step 5: Find the optimal assignment and return it.
    row_ind, col_ind = np.nonzero(mask)
    return list(zip(row_ind.tolist(), col_ind.tolist()))


## Auction algorithm

The auction_algorithm function takes as input the distance_matrix, which contains the distances between each sales representative and each territory, as well as the number of n_reps sales representatives and n_territories territories to be assigned. It also has an optional epsilon parameter that controls the size of the price updates at each iteration.

The function initializes the prices and assignments to zero and the list of unassigned territories to all of the territories. It then enters a loop that iteratively finds the highest bidder for each unassigned territory and assigns that territory to the winning sales representative. The loop continues until all territories have been assigned.

The algorithm finds the highest bidder for each territory by calculating the difference between the distance to each sales representative and the current price for that sales representative. It then selects the sales representative with the highest bid and updates the prices for all sales representatives to be the winning bid plus epsilon. If there are ties for the highest bid, the algorithm randomly selects one of the tying bidders to be the winner.

Finally, the function returns a list of assignments indicating which sales representative is assigned to each territory. Note that the function assumes that each sales representative can only be assigned to one territory and that there are no constraints on which territories can be assigned to which sales representatives. If these assumptions do not hold, the function would need to be modified accordingly.

In [ ]:
import numpy as np

def auction_algorithm(distance_matrix, n_reps, n_territories, epsilon=0.01):
    # initialize variables
    prices = np.zeros(n_reps)
    assignments = np.zeros(n_territories)
    unassigned = np.arange(n_territories)
    
    # start auction iterations
    while len(unassigned) > 0:
        # find bidders and their bids
        bidders = np.where(assignments == 0)[0]
        bids = np.zeros((len(bidders), n_reps))
        for i, bidder in enumerate(bidders):
            bids[i, :] = distance_matrix[bidder, :] - prices
        winning_bids = np.max(bids, axis=1)
        winners = np.argmax(bids, axis=1)
        
        # check for ties and adjust prices
        counts = np.bincount(winners)
        ties = np.where(counts > 1)[0]
        for t in ties:
            indices = np.where(winners == t)[0]
            tie_bids = winning_bids[indices]
            max_bid = np.max(tie_bids)
            winners[indices] = np.where(tie_bids == max_bid)[0][0]
            winning_bids[indices] = max_bid
        prices += winning_bids + epsilon
        
        # assign territories to winners
        for i, bidder in enumerate(bidders):
            if winners[i] != -1:
                assignments[bidder] = winners[i] + 1
                unassigned = np.delete(unassigned, np.where(unassigned == bidder)[0])
    
    return assignments


## Genetic Algorithm

In this example, we define a problem where we have 20 territories and 10 sales representatives, each with a different skill set. The territories and sales representatives are randomly generated, as are the skill sets. We then define the genetic algorithm parameters and functions, including the fitness function, which evaluates the fitness of an individual assignment. The genetic algorithm is then run for a specified number of generations, and the best individual is returned as the solution. The output is the best assignment of sales representatives to territories.

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import manhattan_distances
import random

# define the problem data
num_territories = 20
num_sales_reps = 10
territories = np.random.rand(num_territories, 2) * 100
sales_reps = np.random.rand(num_sales_reps, 2) * 100
skill_sets = np.random.randint(1, 6, num_sales_reps)

# define the genetic algorithm parameters
pop_size = 100
num_generations = 100
mutation_rate = 0.1

# define the fitness function
def fitness(individual):
    dist = manhattan_distances(territories, sales_reps[individual])
    return np.sum(skill_sets[individual] / (dist + 0.1))

# define the genetic algorithm functions
def create_individual():
    return random.sample(range(num_sales_reps), num_territories)

def crossover(parent1, parent2):
    child1 = parent1.copy()
    child2 = parent2.copy()
    crossover_point = random.randint(1, num_territories-1)
    child1[crossover_point:], child2[crossover_point:] = child2[crossover_point:], child1[crossover_point:]
    return child1, child2

def mutate(individual):
    for i in range(num_territories):
        if random.random() < mutation_rate:
            individual[i] = random.randint(0, num_sales_reps-1)
    return individual

# define the genetic algorithm
def genetic_algorithm():
    # initialize the population
    population = [create_individual() for _ in range(pop_size)]
    # evolve the population
    for i in range(num_generations):
        # evaluate the fitness of each individual
        fitness_scores = [fitness(individual) for individual in population]
        # select the parents
        parents = np.random.choice(population, size=pop_size, replace=True, p=fitness_scores/sum(fitness_scores))
        # create the next generation
        offspring = []
        for j in range(0, pop_size, 2):
            parent1, parent2 = parents[j], parents[j+1]
            child1, child2 = crossover(parent1, parent2)
            offspring.append(mutate(child1))
            offspring.append(mutate(child2))
        # replace the old generation with the new generation
        population = offspring
    # return the best individual
    best_individual = max(population, key=fitness)
    return best_individual

# run the genetic algorithm and print the results
best_individual = genetic_algorithm()
print("Best individual:", best_individual)


## Branch and bound algorithm

To use this function for assigning territories to sales representatives, you would need to create a cost matrix that represents the cost of assigning each sales representative to each territory, and a capacities list that represents the number of territories each sales representative can cover. You can modify the code to add additional constraints or considerations as needed.

In [1]:
import numpy as np
from itertools import permutations

def branch_and_bound(cost_matrix, capacities):
    n = len(cost_matrix)
    # initialize lower bound as sum of minimum cost for each row
    lower_bound = np.sum(np.min(cost_matrix, axis=1))
    # initialize upper bound as sum of all costs
    upper_bound = np.sum(cost_matrix)
    # initialize best solution and its cost
    best_solution = None
    best_cost = float('inf')
    # initialize stack with root node
    stack = [(0, [], lower_bound)]
    while stack:
        level, assignment, cost_so_far = stack.pop()
        if level == n:
            # if all assignments have been made, check if it's a valid solution and update best solution if necessary
            if cost_so_far < best_cost:
                best_solution = assignment
                best_cost = cost_so_far
        else:
            # get list of possible assignments at this level
            possible_assignments = [(i, cost_matrix[level][i]) for i in range(n) if cost_matrix[level][i] != float('inf') and capacities[i] > 0]
            # sort by increasing cost
            possible_assignments.sort(key=lambda x: x[1])
            for i, cost in possible_assignments:
                # make a copy of the assignment list and capacities list
                new_assignment = assignment.copy()
                new_capacities = capacities.copy()
                # add the new assignment
                new_assignment.append((level, i))
                # subtract capacity of the sales representative assigned to this territory
                new_capacities[i] -= 1
                # calculate new lower bound and upper bound
                new_lower_bound = cost_so_far + cost + np.sum(np.min(cost_matrix[level+1:], axis=1))
                new_upper_bound = cost_so_far + cost + upper_bound - lower_bound
                # if the new upper bound is lower than the current best cost, there's no need to explore this branch
                if new_upper_bound < best_cost:
                    continue
                # otherwise, add this node to the stack
                stack.append((level+1, new_assignment, cost_so_far + cost))
            # add a node with no assignment at this level
            stack.append((level+1, assignment, cost_so_far))
    # create the solution matrix
    solution_matrix = np.zeros((n, n))
    for i, j in best_solution:
        solution_matrix[i][j] = 1
    return solution_matrix, best_cost


## Lagrangian relaxation algorithm

To use this code for assigning territories to sales representatives based on optimizing multiple conflicting goals, you would need to define the following:

The objective function coefficients c for each sales representative-territory pair, which could include factors such as distance, market potential, sales history, etc.

The equality constraint coefficients A_eq and values b_eq that enforce the constraint that each territory is assigned to exactly one sales representative.

The inequality constraint coefficients A_ub and values b_ub that enforce any additional capacity constraints or other constraints on the assignment.

The weights weights for each objective, which determine the relative importance of each objective in the optimization.

You can then call the lagrangian_relaxation function with these inputs to obtain the optimal assignment of territories to sales representatives that balances multiple conflicting goals.

In [3]:
import numpy as np
from scipy.optimize import linprog

def lagrangian_relaxation(c, A_eq, b_eq, A_ub, b_ub, weights, max_iter=100, rho=1.0, epsilon=0.01):
    """
    Solves a multi-objective linear assignment problem using the Lagrangian relaxation algorithm.
    :param c: array-like, shape (n_features,): objective function coefficients.
    :param A_eq: array-like, shape (n_constraints, n_features): equality constraint coefficients.
    :param b_eq: array-like, shape (n_constraints,): equality constraint values.
    :param A_ub: array-like, shape (n_constraints, n_features): inequality constraint coefficients.
    :param b_ub: array-like, shape (n_constraints,): inequality constraint values.
    :param weights: array-like, shape (n_objectives,): weights for each objective.
    :param max_iter: int, maximum number of iterations to run the algorithm.
    :param rho: float, step size for updating the Lagrange multipliers.
    :param epsilon: float, convergence tolerance for the algorithm.
    :return: array-like, shape (n_features,): optimal solution.
    """
    n_features = len(c)
    n_objectives = len(weights)
    x = np.zeros(n_features)
    u = np.zeros((n_objectives, n_features))
    for i in range(max_iter):
        # solve the relaxed problem
        c_relaxed = c - np.sum(u * weights.reshape(-1, 1), axis=0)
        res = linprog(c_relaxed, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, bounds=(0, 1))
        x_new = res.x
        # update the Lagrange multipliers
        for j in range(n_objectives):
            u[j] += rho * weights[j] * (x_new - x)
            u[j] = np.maximum(u[j], 0)  # project onto the non-negative orthant
        # check for convergence
        if np.linalg.norm(x_new - x) < epsilon:
            break
        x = x_new
    return x
